In [2]:
#Load a model and query
# import libraries and packages
DEBUG=True
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import gym
from stable_baselines3 import PPO
import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from droneenv import DroneEnv
import time
#Start the timer
start_time = time.time()

logdir = f"logs/{int(time.time())}/"

env = DroneEnv()
env.reset()
env = Monitor(env, logdir)
#C:\Users\greg\Documents\GitHub\DATA3402\Exams\Final\Drone\models\1670214614
models_dir = f"models/1670892918/"
model_path = f"{models_dir}/590000.zip"
model = PPO.load(model_path, env=env)
#model = PPO("MlpPolicy", env, verbose=1, tensorboard_log = logdir)
episodes = 1
maxReward=-99
data=list()
log=list()
for ep in range(episodes):
    obs = env.reset()
    done = False
    i=0
    while not done:
        
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        #obs[0]=i
        i+=1
        #if action[0]==1 and reward>-1:
        #if reward >=0:
        #    print("i:{} action: {}   reward:{}  obs:{} done:{}".format(i,action,reward,obs,done) )

        #data.append( (int(action[0]),int(action[1]),int(action[2]),int(action[3]),float(reward)))
        #if 20>reward>0  and False :
        #    #i+=1
        #    print("i:{} action: {}   reward:{}  obs:{} done:{}".format(i,action,reward,obs,done) )
        if reward>0 :
            i+=1
            print("i:{} action: {}   reward:{}  obs:{} done:{}".format(i,action,reward,obs,done) )
            if action[1]==0:
                LUDW='L'
            elif action[1]==1:
                LUDW='U'
            elif action[1]==2:
                LUDW='D'
            else:
                LUDW='W'
            log.append( (str(action[0])+" "+LUDW+" "+str(action[2])+" "+str(action[3])+" 1"))
        #if i==25:
        #   env.showStatus(0)
print("--- %s seconds ---" % (time.time() - start_time))   
#raise SystemExit("Stop right there!") 
# create DataFrame using data
#df = pd.DataFrame(data, columns =['drone', 'LUDW', 'param1','prodNum','reward'])

#Count the total distance traveled & time for each drone
#Remember: loading multiple items onto a drone should only count once

DRONE_QTY=30
env = DroneEnv()
env.reset()
droneDist=[0]*DRONE_QTY
longestDistance=0
for droneNum in range(DRONE_QTY):
    #r,c=startRow,startCol
    elapsedTime=0
    dummy="-1 X -1 -1 -1"
    previousLine=dummy.split()
    prevRow,prevCol=113,179
    currRow,currCol=113,179
    for i in range(len(log)):
        line=log[i].split()
        #print ("Full line:{} ".format(log[i].getLine())   )
        
        #Review the log lines for this drone in succession
        if int(line[0])==(droneNum):
            #Update the current position based on the log action
            if line[1]=="L" or line[1]=="U":
                DEBUG and print("Drone:{} current location: {},{} ".format(droneNum,currRow,currCol ))
                currRow,currCol=env.getWarehouseLoc(int(line[2]))
            elif line[1]=="D":
                orderNum=int(line[3])
                origOrderNum=env.getOrigOrderNum(orderNum)
                cr,cc=env.getOrderLoc(orderNum)
                DEBUG and print("Drone:{} current location: {},{} ".format(droneNum,currRow,currCol ))
                currRow,currCol=(cr,cc)
                #Change the order number in the log to the original order number
                log[i]=str(line[0])+" D "+str(origOrderNum)+" "+str(line[3])+" 1"
            elif line[1]=='W':
                elapsedTime+=int(line[2])
            else:
                print("Line: {} in log does not match L/U/D/W. line[1]=<{}>".format(line,line[1]))
            if (currRow,currCol) != (prevRow,prevCol):
                #tally the distance/time
                math.sqrt(    ( prevRow-currRow   )**2 +  ( prevCol-currCol  )**2     )
                distanceFlown=math.ceil(  math.sqrt( (prevRow-currRow)**2 + (prevCol-currCol)**2 ) )
                elapsedTime+=distanceFlown+1  # 1 unit of time for the load duration
                DEBUG and print("{} Drone:{} moved from {},{} to  {},{} Dist:{}    New total={}".format(line,droneNum,prevRow,prevCol,currRow,currCol,distanceFlown+1,elapsedTime))
                #update previous location to current location
                prevRow,prevCol=currRow,currCol
            #if elapsedTime>calc.currentTime():
            #    print("Error 911: Something went wrong!!")
            #    print("Drone: {} Elapsed time:{} is greater than total sim time:{} Delta:{}".format(droneNum,elapsedTime,calc.currentTime(),calc.currentTime()-elapsedTime))
            #    #raise SystemExit("Stop here!")
    droneDist[droneNum]=elapsedTime
    if elapsedTime>longestDistance:
        DEBUG and print("This drone:{} had the longest distance so far:{}".format(droneNum,elapsedTime) )
        longestDistance=elapsedTime
        longestDrone=droneNum
for droneNum in range(DRONE_QTY):
    print("Drone:{}  had a total distance flown of: {}".format(droneNum,droneDist[droneNum]))
    padding=longestDistance-droneDist[droneNum]
    if padding>longestDistance:
        print("Something is broken: Padding:{} is greater than total sim time{}. Delta:{}".format(padding,calc.currentTime(),calc.currentTime()-padding))
        #print("Longest drone detail:")
        #for i in range(Log.getQty()):
        #    line=log[i].getLine().split()
        #    #print (line)
        #    if int(line[0])==(longestDrone):
        #        print("Line: {} ".format(line)) 
        #raise SystemExit("Error 999:Stop right there!") 
    if padding>0:
        print("Drone {} needs to be padded by: {}".format(droneNum,padding))   
        log.append( (str(droneNum)+" W "+str(padding)))         
        
#Write the output file
#Log.getCount()
outputFile = open("submission.csv", "w")
result=outputFile.write( str( len(log) )+'\n'  )
for i in range(len(log)):
    result=outputFile.write(log[i]+'\n')

outputFile.close()
#raise SystemExit("Stop right there!") 
#print("--- %s seconds ---" % (time.time() - start_time))     



                                            ([ #Drones,LUDW,orders,#products ])
self.action_space = gym.spaces.MultiDiscrete([ 2,3,54,400 ])
                                                ManSeq#,orders_Completed,#drones,Drone@Loc,PayloadWeight,priorCmd,currentCmd ])
self.observation_space = gym.spaces.MultiDiscrete([ 270,55,31,21,202,3,3 ])
Wrapping the env in a DummyVecEnv.
i:2 action: [ 0  0  0 18]   reward:0.25  obs:[ 1  0  0  0 40  3  0] done:False
i:4 action: [ 0  0  0 48]   reward:0.75  obs:[  2   0   0   0 102   0   0] done:False
i:7 action: [ 0  0  0 34]   reward:0.75  obs:[  4   0   0   0 154   0   0] done:False
i:9 action: [ 0  2  0 48]   reward:99.9584045029338  obs:[ 5  0  0 18 92  0  2] done:False
i:13 action: [ 0  0  0 25]   reward:0.25  obs:[ 8  1  0  0 96  2  0] done:False
i:15 action: [ 0  0  0 35]   reward:0.75  obs:[  9   1   0   0 104   0   0] done:False
i:28 action: [ 0  2  0 25]   reward:99.62210048410078  obs:[ 21   1   0  14 100   0   2] done:False
i:31 action

In [6]:
log

['0 L 0 18 1',
 '0 L 0 48 1',
 '0 L 0 34 1',
 '0 D 8 48 1',
 '0 L 0 25 1',
 '0 L 0 35 1',
 '0 D 4 25 1',
 '0 L 0 4 1',
 '0 D 6 35 1',
 '0 D 3 18 1',
 '0 L 0 41 1',
 '0 D 0 4 1',
 '0 L 0 5 1',
 '0 D 7 41 1',
 '0 D 0 5 1',
 '0 L 7 28 1',
 '0 D 6 34 1',
 '0 L 2 26 1',
 '0 D 4 28 1',
 '0 L 0 37 1',
 '0 D 6 37 1',
 '0 L 0 52 1',
 '0 L 0 21 1',
 '0 D 9 52 1',
 '0 L 0 45 1',
 '0 D 4 26 1',
 '0 L 0 16 1',
 '0 D 3 16 1',
 '0 L 7 8 1',
 '0 D 1 8 1',
 '0 L 0 47 1',
 '0 D 8 47 1',
 '0 L 0 49 1',
 '0 D 9 49 1',
 '0 L 0 7 1',
 '0 L 0 10 1',
 '0 D 2 10 1',
 '0 D 4 21 1',
 '0 L 0 22 1',
 '0 D 4 22 1',
 '0 L 0 50 1',
 '0 D 7 45 1',
 '0 L 7 14 1',
 '0 D 9 50 1',
 '0 D 3 14 1',
 '0 L 0 13 1',
 '0 D 0 7 1',
 '0 L 0 40 1',
 '0 D 3 13 1',
 '0 L 0 15 1',
 '0 D 3 15 1',
 '0 L 0 17 1',
 '0 D 3 17 1',
 '0 D 7 40 1',
 '0 L 0 19 1',
 '1 W 9160',
 '2 W 9160',
 '3 W 9160',
 '4 W 9160',
 '5 W 9160',
 '6 W 9160',
 '7 W 9160',
 '8 W 9160',
 '9 W 9160',
 '10 W 9160',
 '11 W 9160',
 '12 W 9160',
 '13 W 9160',
 '14 W 916